In [4]:
data_file = "/media/stormfield/Data/resources/lastfm/userid-timestamp-artid-artname-traid-traname.tsv"

In [5]:
import pandas as pd

In [7]:
raw_df = pd.read_csv(data_file,
                     low_memory=False, 
                     sep='\t', names=['user_id','timestamp','artist_id','artist', 'track_id','track'],
                     parse_dates=['timestamp'])

since it took a while to read. lets quickly check the memory and see of there are any improvemnts we can make

## Data exploration

First lets gets some information regarding the dtypes and memeory involed

In [10]:
raw_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19098862 entries, 0 to 19098861
Data columns (total 6 columns):
user_id      object
timestamp    datetime64[ns, UTC]
artist_id    object
artist       object
track_id     object
track        object
dtypes: datetime64[ns, UTC](1), object(5)
memory usage: 7.1 GB


Since we are time bound, we will optimize for memory if required later. For now this can be easily handled by most morden machines.

lets take a look at how the data looks like.

In [15]:
raw_df.sample(5)

,user_id,timestamp,artist_id,artist,track_id,track
13330652,user_000704,2007-06-06 00:25:51+00:00,172e1f1a-504d-4488-b053-6344ba63e6d0,Nick Cave & The Bad Seeds,b41dcee8-716b-4e24-b34e-2d603e30b4a3,Deanna
5349985,user_000274,2007-10-11 20:02:14+00:00,f4c967a4-7591-4bba-8237-82e171a2fa7f,X-Ray Spex,1c6df6d6-d89e-44ed-9dca-0bb5d7260760,Oh Bondage Up Yours!
16655117,user_000861,2007-03-07 12:02:51+00:00,2865730a-5757-4005-875e-84d6274cb1c0,Revolting Cocks,10267304-f269-41a8-a2c5-a026f0a69bed,You Often Forget (Malignant)
18288441,user_000950,2007-04-07 17:31:29+00:00,NaN,Consul The Educated Monkey,NaN,Poema De La Tormenta
14013489,user_000743,2007-12-20 21:47:07+00:00,006f0783-c5a0-458b-a9da-f8551f7ebe77,Richard Hawley,4f45ddfd-53d5-471c-83e2-4d976b818211,The Sun Refused To Shine


### Statistics of each column
<a id='staticstics_cell'></a>

In [20]:
raw_df.describe()

,user_id,timestamp,artist_id,artist,track_id,track
count,19098862,19098862,18498009,19098862,16936136,19098850
unique,992,17454739,107295,173923,960402,1083480
top,user_000949,2009-02-26 21:29:15+00:00,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,db16d0b3-b8ce-4aa8-a11a-e4d53cc7f8a6,Intro
freq,183103,248,115099,115099,3991,17561
first,NaN,2005-02-14 00:00:07+00:00,NaN,NaN,NaN,NaN
last,NaN,2013-09-29 18:32:04+00:00,NaN,NaN,NaN,NaN


#### 1. Check for nulls

In [21]:
raw_df.isnull().sum()

user_id            0
timestamp          0
artist_id     600853
artist             0
track_id     2162726
track             12
dtype: int64

In [23]:
raw_df[raw_df['artist_id'].isnull()].head()

,user_id,timestamp,artist_id,artist,track_id,track
50,user_000001,2009-05-02 14:30:56+00:00,NaN,Rocket Empire,NaN,Simmer Down Jammie
78,user_000001,2009-05-01 11:58:15+00:00,NaN,Rocket Empire,NaN,Simmer Down Jammie
87,user_000001,2009-05-01 09:36:10+00:00,NaN,Rocket Empire,NaN,Simmer Down Jammie
92,user_000001,2009-05-01 09:12:21+00:00,NaN,Nicken,NaN,Lifeb
95,user_000001,2009-05-01 08:58:19+00:00,NaN,Ivan Enot,NaN,Grooves Of The Heart


As we can see above the artist_id & track_id have lot of nulls. Since the corresponding artist/track columns aren't null its best to drop those 'id' columns and proceed

In [24]:
df = raw_df.drop(columns=['artist_id', 'track_id'])

Thats much cleaner isn't it ? (PFB)

In [26]:
df[df['artist'] == 'Rocket Empire'].head()

,user_id,timestamp,artist,track
50,user_000001,2009-05-02 14:30:56+00:00,Rocket Empire,Simmer Down Jammie
78,user_000001,2009-05-01 11:58:15+00:00,Rocket Empire,Simmer Down Jammie
87,user_000001,2009-05-01 09:36:10+00:00,Rocket Empire,Simmer Down Jammie
102,user_000001,2009-05-01 08:18:35+00:00,Rocket Empire,Simmer Down Jammie
117,user_000001,2009-05-01 05:59:11+00:00,Rocket Empire,Simmer Down Jammie


### 2. Combine columns to get unique workable data

When we pulled [staticstics](#staticstics_cell), we observerd that there is a lot of 'intro' tracks. This shouldn't be a suprising. To get unique track details we need to combine the artist & track columns

Based on [Last Fm](https://www.last.fm/tag/sampling) and general consensus we can combine both columns using an aceeptable format, ie `Artist_name - Track_name`

In [28]:
df['song'] = df['artist'] + ' - ' + df['track']

lets see if the data came correct

In [31]:
df.sample(5)

,user_id,timestamp,artist,track,song
11742312,user_000628,2006-08-16 14:34:33+00:00,Talking Heads,Heaven (Live),Talking Heads - Heaven (Live)
17502363,user_000909,2007-03-14 04:50:23+00:00,Radiohead,Just,Radiohead - Just
7770403,user_000412,2007-03-14 16:14:28+00:00,Tunng,Sweet William,Tunng - Sweet William
4406585,user_000229,2007-06-22 08:40:06+00:00,Metallica,Phantom Lord,Metallica - Phantom Lord
9223762,user_000496,2006-06-10 19:26:20+00:00,Pantera,Throes Of Rejection,Pantera - Throes Of Rejection


We ca drop the artist & track columns for now, since we will use the song colums for further analysis

In [35]:
df.drop(columns=['artist', 'track'],inplace=True)

Lets sample the data one last time

In [37]:
df.sample(5)

,user_id,timestamp,song
2329584,user_000119,2007-03-08 17:14:57+00:00,Black Label Society - Give Yourself To Me
15904910,user_000824,2008-05-08 18:18:55+00:00,Reuben - Freddy Kreuger
16790440,user_000869,2008-04-26 10:06:06+00:00,Nightmares On Wax - Damn
12324500,user_000667,2006-07-05 12:38:51+00:00,Cheap Trick - I Want You To Want Me (Live)
156029,user_000006,2006-10-08 20:58:46+00:00,Metallica - Turn The Page


### Part A


Create a list of user IDs, along with the number of distinct songs each user has played

In [83]:
distinct_songs_by_user = df.groupby('user_id')['song'].nunique().reset_index()

In [102]:
# quick rename of the result column
distinct_songs_by_user.rename(columns={'song':'count'},inplace=True)

In [104]:
distinct_songs_by_user.head()

,user_id,count
0,user_000001,3143
1,user_000002,8537
2,user_000003,4682
3,user_000004,6210
4,user_000005,2014


 quick verfication

quick sanity check reveals that user_000982 occurs 54 times. Yes I did manually go through the 55 entries as well to see if they are unique. But just incase lets try verifying it another way

In [100]:
distinct_songs_by_user[distinct_songs_by_user['user_id']=='user_000982']

,user_id,count
974,user_000982,54


In [101]:
df[df['user_id']=='user_000982'].nunique()

user_id       1
timestamp    55
song         54
dtype: int64